In [9]:
import pandas as pd
import numpy as np

In [41]:
df= pd.read_csv(r'letras.txt', sep='\t', skiprows=0, header=None)
df.head(12)

,0,1,2,3,4
0,1.0,1.0,1.0,1.0,1.0
1,1.0,-1.0,-1.0,-1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,-1.0,-1.0,-1.0,1.0
4,1.0,-1.0,-1.0,-1.0,1.0
5,NaN,NaN,NaN,NaN,NaN
6,1.0,1.0,1.0,1.0,1.0
7,1.0,-1.0,-1.0,-1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,1.0,-1.0,-1.0,-1.0,1.0


Paso las matrices a vectores 

In [42]:
dfToRows= []
toAdd= []
for index, row in df.iterrows():
    for column in df:
        number= row[column]
        if (not pd.isna(number)):
            toAdd.append(int(number))
        else:
            dfToRows.append(toAdd)
            toAdd=[]
            break

print(dfToRows[0])
print(dfToRows[1])

[1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1]
[1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1]


In [49]:
product= np.dot(dfToRows[11], dfToRows[23])
print(product)

1


In [50]:
product= np.dot(dfToRows[14], dfToRows[16])
print(product)

23
